# Projects -Cohort Analysis for assessing customer retention in E-commerce industry

## 04 - Cohort Analysis - cohort segmentation retention

Why do Cohort Analysis?
• Understand customer retention over time.
• See how many customers come back after their first purchase.
• Check if newer customers behave better or worse than older ones.
• Identify customer churn problems or marketing successes.